# DEMO Part 1: Data Preparation, Loading, Feature Engineering

In [1]:
import pandas as pd
import Technic as tc
from Technic import TSFM, DumVar

## Prepare and Load Data

In [2]:
df_internal = pd.read_csv(r'Demo Data/housing_market.csv')
df_internal.head()

,date,home_price_index,new_permits,housing_starts,housing_completions,months_supply,new_home_sales,mortgage_rate,construction_ppi,rent_cpi
0,2000-01-01,100.000,1727,1636,1574,4.3,873,8.06,144.1,180.9
1,2000-02-01,100.571,1692,1737,1677,4.3,856,8.25,144.7,181.3
2,2000-03-01,101.466,1651,1604,1704,4.3,900,8.31,145.4,181.9
3,2000-04-01,102.540,1597,1626,1610,4.4,841,8.23,145.6,182.3
4,2000-05-01,103.701,1543,1575,1682,4.4,857,8.13,144.9,182.8


In [ ]:
df_internal['home_price_GR1'] = df_internal['home_price_index'].pct_change().shift(-1)
df_internal['home_price_GR3'] = df_internal['home_price_index'].pct_change(3).shift(-1)

In [13]:
ID_config = {
    'in_sample_start':"2006-01-31",
    'in_sample_end':"2023-09-30",
    'full_sample_end':"2025-09-30",
}
int_ldr = tc.TimeSeriesLoader(
    in_sample_start="2006-01-31",
    in_sample_end="2023-09-30",
    full_sample_end="2025-09-30",
)
int_ldr.load(df_internal, date_col='date')

In [14]:
int_ldr.internal_data

,home_price_index,new_permits,housing_starts,housing_completions,months_supply,new_home_sales,mortgage_rate,construction_ppi,rent_cpi,home_price_GR,home_price_GR3
date,,,,,,,,,,,
2000-01-31,100.000,1727,1636,1574,4.3,873,8.06,144.100,180.900,NaN,NaN
2000-02-29,100.571,1692,1737,1677,4.3,856,8.25,144.700,181.300,NaN,NaN
2000-03-31,101.466,1651,1604,1704,4.3,900,8.31,145.400,181.900,NaN,NaN
2000-04-30,102.540,1597,1626,1610,4.4,841,8.23,145.600,182.300,0.025400,0.025400
2000-05-31,103.701,1543,1575,1682,4.4,857,8.13,144.900,182.800,0.031122,0.031122
...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30,328.978,1415,1306,1371,7.9,738,6.56,341.038,438.322,-0.007988,-0.007988
2025-10-31,328.442,1412,1246,1386,7.9,737,6.30,340.292,438.322,-0.007686,-0.007686
2025-11-30,328.442,1412,1246,1386,7.9,737,6.17,338.588,438.875,-0.004374,-0.004374


## MEV Loader

In [16]:
# Read Macro Data
df_mev_qtr = pd.read_csv(r'Demo Data/macro_quarterly.csv')
df_mev_mth = pd.read_csv(r'Demo Data/macro_monthly.csv')
df_mev_mth.ffill(inplace=True)

In [17]:
# Create loader for Macro Variables
mev_ldr = tc.MEVLoader()

# load quarterly macro data
mev_ldr.load(
    source=df_mev_qtr,
    date_col='observation_date'
)

# load monthly macro data
mev_ldr.load(
    source=df_mev_mth,
    date_col='observation_date'
)

## Data Manager

In [18]:
dm = tc.DataManager(
    int_ldr,
    mev_ldr
)

In [19]:
# Access mev dataframe used for model training
dm.model_mev.head()

,USFF,USIORB,USGOV6M,USGOV1Y,USGOV2Y,USGOV5Y,USGOV10Y,USGOV30Y,USGOVR10Y,USSOFR,...,USVIXA,USRGDP,USNGDP,USCP,M,USPGDP,Q,USHHPDI,USNCREIF,USGDPDEF
observation_date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,5.45,NaN,5.74,5.98,6.24,6.36,6.45,6.48,NaN,NaN,...,24.64,4598.488524,3307.231843,189.680068,1,4534.237451,1,NaN,123381.094909,71.921004
2000-02-29,5.73,NaN,5.95,6.30,6.60,6.68,6.62,6.43,NaN,NaN,...,23.45,4623.037165,3331.782425,187.920065,2,4549.673408,1,NaN,124436.965995,72.068840
2000-03-31,5.85,NaN,6.02,6.17,6.52,6.59,6.39,6.16,NaN,NaN,...,21.64,4656.621311,3363.164732,187.442867,3,4566.080901,1,NaN,127553.939097,72.223157
2000-04-30,6.02,NaN,6.15,6.28,6.50,6.32,6.03,5.84,NaN,NaN,...,24.11,4705.192291,3405.413944,186.415347,4,4582.196126,2,NaN,129965.047322,72.374916
2000-05-31,6.27,NaN,6.25,6.24,6.73,6.59,6.29,5.98,NaN,NaN,...,25.88,4713.926434,3418.653937,186.630677,5,4597.873265,2,NaN,133539.540908,72.523095


In [28]:
# Feature Engineering Function
def new_features(df_mev: pd.DataFrame, df_in: pd.DataFrame):
    # -------------------------------------------------------------------------
    # A) Rates & curve shape (term structure signals)
    # -------------------------------------------------------------------------
    df_mev['USYC10_2'] = df_mev['USGOV10Y'] - df_mev['USGOV2Y']
    df_mev['USYC10_1'] = df_mev['USGOV10Y'] - df_mev['USGOV1Y']
    df_mev['USYC10_6M'] = df_mev['USGOV10Y'] - df_mev['USGOV6M']
    df_mev['USYC30_10'] = df_mev['USGOV30Y'] - df_mev['USGOV10Y']
    df_mev['USYC5_2'] = df_mev['USGOV5Y'] - df_mev['USGOV2Y']
    df_mev['USREAL_TERM_PREM_PROXY'] = df_mev['USGOV10Y'] - df_mev['USGOVR10Y']

    # -------------------------------------------------------------------------
    # B) Credit conditions (spreads / wedges)
    # -------------------------------------------------------------------------
    df_mev['USCORP_SPRD_BAA_AAA'] = df_mev['USCORPBBB10Y'] - df_mev['USCORPAA10Y']
    df_mev['USCORP_SPRD_BAA_T10'] = df_mev['USCORPBBB10Y'] - df_mev['USGOV10Y']
    df_mev['USCORP_SPRD_AAA_T10'] = df_mev['USCORPAA10Y'] - df_mev['USGOV10Y']
    df_mev['USCP_FF_SPRD'] = df_mev['USCPF'] - df_mev['USFF']
    df_mev['USPRIME_FF_SPRD'] = df_mev['USPRIME'] - df_mev['USFF']
    df_mev['USIORB_FF_SPRD'] = df_mev['USIORB'] - df_mev['USFF']
    df_mev['USSOFR_FF_SPRD'] = df_mev['USSOFR'] - df_mev['USFF']

    # -------------------------------------------------------------------------
    # C) Mortgage pricing & household financing wedges
    # -------------------------------------------------------------------------
    df_mev['USMORT30_T10_SPRD'] = df_mev['USMORT30Y'] - df_mev['USGOV10Y']
    df_mev['USMORT15_T10_SPRD'] = df_mev['USMORT15Y'] - df_mev['USGOV10Y']
    df_mev['USARM_T2_SPRD'] = df_mev['USAM51'] - df_mev['USGOV2Y']
    df_mev['USMORT30_15_SPRD'] = df_mev['USMORT30Y'] - df_mev['USMORT15Y']

    # -------------------------------------------------------------------------
    # D) Liquidity / leverage ratios
    # -------------------------------------------------------------------------
    df_mev['USLIQ_M2_GDP'] = df_mev['USM2'] / df_mev['USNGDP']
    df_mev['USLIQ_M1_M2'] = df_mev['USM1'] / df_mev['USM2']
    df_mev['USCREDIT_CC_INC'] = df_mev['USCC'] / df_mev['USDI']
    df_mev['USCREDIT_CC_GDP'] = df_mev['USCC'] / df_mev['USNGDP']
    df_mev['USCONS_PCE_INC'] = df_mev['USNC'] / df_mev['USDI']
    df_mev['USSAV_DLR'] = df_mev['USDI'] - df_mev['USNC']

    # -------------------------------------------------------------------------
    # E) Nominal vs real “price-level wedges” (implied deflators)
    # -------------------------------------------------------------------------
    df_mev['USPCE_IMPLICIT_DEF'] = df_mev['USNC'] / df_mev['USRC']
    df_mev['USDPI_IMPLICIT_DEF'] = df_mev['USDI'] / df_mev['USRPDI']

    # -------------------------------------------------------------------------
    # F) Risk appetite & relative valuation proxies
    # -------------------------------------------------------------------------
    df_mev['USRISKON_SPX_VIX'] = df_mev['USSP500'] / df_mev['USVIXA']

    # -------------------------------------------------------------------------
    # G) Real estate cross-market relative pricing
    # -------------------------------------------------------------------------
    df_mev['USCRE_HOUS_REL'] = df_mev['USNCREIF'] / df_mev['USCSH']
    df_mev['USCP_GDP_SHARE'] = df_mev['USCP'] / df_mev['USNGDP']

    return df_mev, df_in

dm.apply_to_all(new_features)

var_update_dict = {
    'USYC10_2': {'type': 'rate', 'category': 'yield slope' },
    'USYC10_1': {'type': 'rate', 'category': 'yield slope' },
}


var_update_dict = {
    # A) Rates & curve shape
    'USYC10_2': {'type': 'rate', 'category': 'yield slope'},
    'USYC10_1': {'type': 'rate', 'category': 'yield slope'},
    'USYC10_6M': {'type': 'rate', 'category': 'yield slope'},
    'USYC30_10': {'type': 'rate', 'category': 'yield slope'},
    'USYC5_2': {'type': 'rate', 'category': 'yield slope'},
    'USREAL_TERM_PREM_PROXY': {'type': 'rate', 'category': 'real rate spread'},

    # B) Credit conditions
    'USCORP_SPRD_BAA_AAA': {'type': 'rate', 'category': 'credit spread'},
    'USCORP_SPRD_BAA_T10': {'type': 'rate', 'category': 'credit spread'},
    'USCORP_SPRD_AAA_T10': {'type': 'rate', 'category': 'credit spread'},
    'USCP_FF_SPRD': {'type': 'rate', 'category': 'funding spread'},
    'USPRIME_FF_SPRD': {'type': 'rate', 'category': 'bank pricing spread'},
    'USIORB_FF_SPRD': {'type': 'rate', 'category': 'policy spread'},
    'USSOFR_FF_SPRD': {'type': 'rate', 'category': 'funding spread'},

    # C) Mortgage wedges
    'USMORT30_T10_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USMORT15_T10_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USARM_T2_SPRD': {'type': 'rate', 'category': 'mortgage spread'},
    'USMORT30_15_SPRD': {'type': 'rate', 'category': 'mortgage spread'},

    # D) Liquidity / leverage ratios
    'USLIQ_M2_GDP': {'type': 'level', 'category': 'liquidity ratio'},
    'USLIQ_M1_M2': {'type': 'level', 'category': 'liquidity ratio'},
    'USCREDIT_CC_INC': {'type': 'level', 'category': 'leverage ratio'},
    'USCREDIT_CC_GDP': {'type': 'level', 'category': 'leverage ratio'},
    'USCONS_PCE_INC': {'type': 'level', 'category': 'consumption ratio'},
    'USSAV_DLR': {'type': 'level', 'category': 'savings level'},

    # E) Implied deflators
    'USPCE_IMPLICIT_DEF': {'type': 'level', 'category': 'price level'},
    'USDPI_IMPLICIT_DEF': {'type': 'level', 'category': 'price level'},

    # F) Risk / relative valuation
    'USRISKON_SPX_VIX': {'type': 'level', 'category': 'risk appetite'},

    # G) Real estate cross-market
    'USCRE_HOUS_REL': {'type': 'level', 'category': 'relative valuation'},
    'USCP_GDP_SHARE': {'type': 'level', 'category': 'income share'},
}

dm.update_var_map(var_update_dict)

In [ ]:
# Build Segment for Home Price Growth
seg_config = {
    'segment_id': 'home_price_GR1',
    'target': 'home_price_GR1',
    'model_type': tc.Growth,
    'target_base': 'home_price_index',
    'data_manager': dm,
    'model_cls': tc.OLS
}

seg = tc.Segment(**seg_config)